# Libraries

In [15]:
from rembg import remove
import requests
from PIL import Image, ImageFilter, ImageEnhance
from io import BytesIO
import os

# Creating directories for original and masked images

In [16]:
os.makedirs('original', exist_ok=True)
os.makedirs('masked', exist_ok=True)

# Image URLs

In [17]:
##img_url = 'https://p.imgci.com/db/PICTURES/CMS/357400/357408.jpg'
img_url = 'https://images.alphacoders.com/126/1267661.jpg'
background_url = 'https://sportsfields.info/wp-content/uploads/2023/04/Real-Madrid.jpg'

In [18]:
img_name = 'RashfordImage.jpg'

# Downloading and saving the original image

In [19]:
img = Image.open(BytesIO(requests.get(img_url).content))
img.save(f'original/{img_name}', format='JPEG')

# Removing background

In [20]:
input_image = open(f'original/{img_name}', 'rb').read()
subject = remove(input_image, alpha_matting=True) 

# Saving the masked image

In [21]:
output_path = f'masked/{img_name}'
with open(output_path, 'wb') as f:
    f.write(subject)

# Loading and resizing the background image to match the foreground

In [22]:
background_img = Image.open(BytesIO(requests.get(background_url).content))
background_img = background_img.resize((img.width, img.height))

In [23]:
foreground_image = Image.open(output_path) # Open the masked image as a PIL image

# Post-processing

## 1. Smoothing

In [24]:
foreground_image = foreground_image.filter(ImageFilter.GaussianBlur(radius=1))  # Adjust radius as needed

## 2. Sharpening

In [25]:
sharpened_image = foreground_image.filter(ImageFilter.SHARPEN)

## 3. Color Correction (Adjusting brightness and contrast)

In [26]:
enhancer = ImageEnhance.Brightness(sharpened_image)
brightened_image = enhancer.enhance(1.1)  # Adjust factor as needed
enhancer = ImageEnhance.Contrast(brightened_image)
final_foreground = enhancer.enhance(1.2)  # Adjust factor as needed

# Pasting the refined foreground onto the background

In [27]:
background_img.paste(final_foreground, (0, 0), final_foreground)

# Saving the final image with the new background

In [28]:
background_img.save('masked/final_image.jpg', format='JPEG')